# 트레이닝 코드

- 네트워크 구성 및 weight 초기화 함수


파이썬 리스트로 MLP 네트워크 구성 (Weight(가중치)를 리스트로 표현한것)

In [1]:
#Training
import math
import random
random.seed(1) #Random을 사용했지만 무작위로 나오지 않게 잡아줌

def MLP2(Input, hidden, output):  # 네트워크 구성 및 weight 초기화
    net = list()
    h_layer = [[random.random() for i in range(Input + 1)] for i in range(hidden)]
    net.append(h_layer)
    o_layer = [[random.random() for i in range(hidden + 1)] for i in range(output)]
    net.append(o_layer)
    return net

In [2]:
from pprint import pprint
net = MLP2(2, 2, 2) #인풋을 2개, 은닉노드 2개, 아웃풋 2개
random.seed(1)

pprint(net)
print()
pprint(net[0])
print()
pprint(net[0][0])
#가중치가 3개인 이유는 2개는 히든노드에 대한 웨이트이고, 나머지 1개는 바이어스에 대한 웨이트임.(여기서 바이어스에 대한 웨이트는 1을 입력으로 받음)

[[[0.13436424411240122, 0.8474337369372327, 0.763774618976614],
  [0.2550690257394217, 0.49543508709194095, 0.4494910647887381]],
 [[0.651592972722763, 0.7887233511355132, 0.0938595867742349],
  [0.02834747652200631, 0.8357651039198697, 0.43276706790505337]]]

[[0.13436424411240122, 0.8474337369372327, 0.763774618976614],
 [0.2550690257394217, 0.49543508709194095, 0.4494910647887381]]

[0.13436424411240122, 0.8474337369372327, 0.763774618976614]


- 시그모이드 함수(활성함수)

In [3]:
#시그모이드함수 -> 출력값을 이진값만으로 반환. 0이냐 1이냐 둘중하나
def Act_sigmoid(x):  # 활성화 함수 - 시그모이드 
    y = 1 / (1 + math.exp(-x))
    return y

- Weight값 통합 함수

In [4]:
def Weight_summation(data, net, layer):
    # 입력값에 layer를 사용한 이유는 히든층에서의 웨이트 계산할 때는 [0]네트워크(웨이트)
    #  아웃층에서 웨이트 계산할 때는 [1]네트워크(웨이트) 사용
    numberOfinp = len(net[0][0]) - 1  # 만약 인풋이 두개면 2
    numberOfhid = len(net[0])  # 히든노드 두개면 2
    numberOfout = len(net[1])  # 아웃노드 두개면 2
    W_s = []  # 웨이트 써메이션 초기화

    if layer == 'H':  # 인풋- > '히든'레이어 갈 때
        for i in range(0, numberOfhid):
            h_w = 0
            h_w = h_w + (1 * net[0][i][numberOfinp])  # 바이어스와 웨이트의 곱 (바이어스노드에 들어오는 입력 값 = 1)
            for j in range(0, numberOfinp):
                h_w = h_w + (data[j] * net[0][i][j])
            W_s.append(h_w)
        W_s = [W_s]

    if layer == 'O':
        for i in range(0, numberOfout):
            o_w = 0
            o_w = o_w + (1 * net[1][i][numberOfhid])
            for j in range(0, numberOfhid):
                o_w = o_w + data[j] * net[1][i][j]  # 히든노드
            W_s.append(o_w)
        W_s = [W_s]  # [ [ ~~ ]]
    return W_s

- 순방향 전파 함수

In [5]:
def Forward_propagation(data, net):  # 순전파 진행하면서 인풋데이타, 인풋-히든 웨이트
    node = list()
    node.append(data)
    Ws1 = Weight_summation(data, net, 'H')  # [[ h1_1_before,  h1_2_before ]]
    # print('[[h1_1_before,  h1_2_before]] : ',Ws1,'\n')
    numberOfhid = len(net[0])  # 히든노드 두개면 2
    numberOfout = len(net[1])

    H1 = list()
    for i in range(0, numberOfhid):
        H1.append(Act_sigmoid(Ws1[0][i]))
    # H1 = [ H1_1 H1_2 ~~~~]
    node.append(H1)

    Ws2 = Weight_summation(H1, net, 'O')  # [[ o1_1_before, o_2_before ]]
    # print('[[ o1_1_before, o_2_before ]] : ',Ws2,'\n')

    O1 = list()
    for i in range(0, numberOfout):
        O1.append(Act_sigmoid(Ws2[0][i]))

    node.append(O1)

    # print("[[IN1,IN2, T], [H1, H2], [OUT1, OUT2]]:",node,'\n')
    return node

- 역방향 전파 함수

In [6]:
def Backward_propagation(weight, dataset):  # 역전파
    numberOfout = len(weight[1])  # 아웃노드 두개면 2
    numberOfdata = len(dataset)
    numberOfhid = len(weight[0])
    numberOfinp = len(weight[0][0]) - 1

    c2 = list()
    h_layer = [[0 for i in range(numberOfinp + 1)] for i in range(numberOfhid)]
    c2.append(h_layer)
    o_layer = [[0 for i in range(numberOfhid + 1)] for i in range(numberOfout)]
    c2.append(o_layer)
    c2

    init_back = c2

    for i in range(0, numberOfdata):
        Data = dataset[i]
        forward = Forward_propagation(Data, weight)
        Output = forward[2]
        Hid = forward[1]
        Input = forward[0]

        table = []
        for i in range(0, numberOfout):
            if i == Data[numberOfinp]:
                table.append(1)
            else:
                table.append(0)

        # 아웃에서 히든으로 역전파 과정
        for i in range(0, numberOfout):
            init_back[1][i][numberOfhid] = init_back[1][i][numberOfhid] + \
                                           ((Output[i] - table[i]) * Output[i] * (1 - Output[i]) * 1)
            # print('init_back[1][',i,'][',numberOfhid,']=',init_back[1][i][numberOfhid])

            for j in range(0, numberOfhid):
                init_back[1][i][j] = init_back[1][i][j] + \
                                     (Output[i] - table[i]) * Output[i] * (1 - Output[i]) * Hid[j]

        # 히든에서 인풋으로 역전파 과정
        for i in range(0, numberOfhid):
            for j in range(0, numberOfout):
                init_back[0][i][numberOfinp] = init_back[0][i][numberOfinp] + \
                                               ((Output[j] - table[j]) * Output[j] * (1 - Output[j]) * weight[1][j][i] \
                                                * Hid[i] * (1 - Hid[i]) * 1)  # 히든 -> 바이어스 역전파
                for k in range(0, numberOfinp):
                    init_back[0][i][k] = init_back[0][i][k] + \
                                         ((Output[j] - table[j]) * Output[j] * (1 - Output[j]) * weight[1][j][i] \
                                          * Hid[i] * (1 - Hid[i]) * Input[k])

    for i in range(0, numberOfhid):
        for j in range(0, numberOfinp + 1):
            init_back[0][i][j] = init_back[0][i][j] / numberOfdata

    for i in range(0, numberOfout):
        for j in range(0, numberOfhid + 1):
            init_back[1][i][j] = init_back[1][i][j] / numberOfdata

    return init_back

- Weight Update 함수

In [7]:
def Weight_update(backpropa, Now_weight):
    numberOfout = len(Now_weight[1])  # 아웃노드 두개면 2
    numberOfhid = len(Now_weight[0])
    numberOfinp = len(Now_weight[0][0]) - 1

    New_weight = list()
    learning_rate = 0.5

    in_hid_w = Now_weight[0]
    hid_out_w = Now_weight[1]

    out_hid_back = backpropa[1]
    hid_in_back = backpropa[0]

    c2 = list()
    h_layer = [[0 for i in range(numberOfinp + 1)] for i in range(numberOfhid)]
    c2.append(h_layer)
    o_layer = [[0 for i in range(numberOfhid + 1)] for i in range(numberOfout)]
    c2.append(o_layer)

    # inp -> hid 업데이트
    for i in range(0, numberOfhid):
        c2[0][i][numberOfinp] = in_hid_w[i][numberOfinp] \
                                - (hid_in_back[i][numberOfinp] * learning_rate)
        for j in range(0, numberOfinp):
            c2[0][i][j] = in_hid_w[i][j] - (hid_in_back[i][j] * learning_rate)

    # hid -> out 업데이트
    for i in range(0, numberOfout):
        c2[1][i][numberOfhid] = hid_out_w[i][numberOfhid] \
                                - (out_hid_back[i][numberOfhid] * learning_rate)
        for j in range(0, numberOfhid):
            c2[1][i][j] = hid_out_w[i][j] - (out_hid_back[i][j] * learning_rate)

    return c2

- Epoch(시행횟수) 함수

In [8]:
def Epoch(dataset, net, N):  # N = 학습횟수

    updatedW = net

    for i in range(0, N):
        bp = Backward_propagation(updatedW, dataset)
        updatedW = Weight_update(bp, updatedW)
        mse = Loss_function(dataset, updatedW)
        print(i + 1, '번 학습함, mse =', mse)
    return updatedW

- 손실함수

In [9]:
def Loss_function(dataset, net):  # 손실함수 - MSE(평균 제곱 오차)
    numberOfdata = len(dataset)
    numberOfinp = len(net[0][0]) - 1
    numberOfout = len(net[1])
    mse = 0

    for j in range(0, numberOfdata):
        Data = dataset[j]
        table = []
        for i in range(0, numberOfout):
            if i == Data[numberOfinp]:
                table.append(1)
            else:
                table.append(0)

        Fpropa = Forward_propagation(Data, net)  # 아웃풋 구하기

        for j in range(0, numberOfout):
            mse = mse + (1 / numberOfout) * (Fpropa[2][j] - table[j]) ** 2

    mse = mse / numberOfdata
    return mse

- Weight 학습

In [10]:
#데이터셋은 단순히 무작위로 입력하면 안됨.
#예를들어 멍멍 했을때 강아지라는 답을 줘야하는데, 멍멍했는데도 고양이라는 답으로 학습을 시키면 결국 쓸모없는 학습을 하게됨.
#그래서 아래 데이터는 단순 랜덤으로 만든 데이터셋이 아니라, 무언가 0이냐 1이냐를 판단하는 기준이 명확히 드러나있지는 않지만
#보이지 않게 녹아있고 그 녹아있는 것을 우리가 학습시키게 될이라고 출력을 내보내는 입 것임. 따라서 단순히 랜덤수를 학습시키는 것은 무의미함.
#0이라고 출력을 내보내는 입력에 대한 경향성이나 규칙, 1력에 대한 경향성의 차이가 존재해야 비로소
#학습에 의미가 생김.

#아래 데이터셋은 교수님이 주신 데이터셋.
data = [[3.5064385449265267, 2.34547092892632525, 0], [4.384621956392097, 3.4530853889904205, 0],
        [4.841442919897487, 4.02507852317520154, 0], [3.5985868973088437, 4.1621314217538705, 0],
        [2.887219775424049, 3.31523082529190005, 0], [9.79822645535526, 1.1052409596099566, 1],
        [7.8261241795117422, 0.6711054766067182, 1], [2.5026163932400305, 5.800780055043912, 1],
        [5.032436157202415, 8.650625621472184, 1], [4.095084253434162, 7.69104329159447, 1]]

net = MLP2(2, 2, 2)
print('초기 가중치:', net, '\n\n')
updatedW = Epoch(data, net, 10000)
print('\n\n최종 업데이트된 가중치:', updatedW)

초기 가중치: [[[0.13436424411240122, 0.8474337369372327, 0.763774618976614], [0.2550690257394217, 0.49543508709194095, 0.4494910647887381]], [[0.651592972722763, 0.7887233511355132, 0.0938595867742349], [0.02834747652200631, 0.8357651039198697, 0.43276706790505337]]] 


1 번 학습함, mse = 0.3322992710825444
2 번 학습함, mse = 0.32555476486561424
3 번 학습함, mse = 0.31887996967970483
4 번 학습함, mse = 0.3123435487084679
5 번 학습함, mse = 0.3060131054312055
6 번 학습함, mse = 0.29995182451848257
7 번 학습함, mse = 0.2942152928636873
8 번 학습함, mse = 0.2888488469662601
9 번 학습함, mse = 0.2838857274400029
10 번 학습함, mse = 0.2793462063830673
11 번 학습함, mse = 0.2752377135752605
12 번 학습함, mse = 0.2715558529592634
13 번 학습함, mse = 0.2682860982617711
14 번 학습함, mse = 0.2654059022736409
15 번 학습함, mse = 0.2628869510796255
16 번 학습함, mse = 0.26069733305113363
17 번 학습함, mse = 0.25880345598800264
18 번 학습함, mse = 0.2571716164670947
19 번 학습함, mse = 0.25576918894309847
20 번 학습함, mse = 0.25456545020163074
21 번 학습함, mse = 0.253532085031274
22

265 번 학습함, mse = 0.19957638938196504
266 번 학습함, mse = 0.19945636374658768
267 번 학습함, mse = 0.19933664809474955
268 번 학습함, mse = 0.1992172364270895
269 번 학습함, mse = 0.19909812275660282
270 번 학습함, mse = 0.19897930110814968
271 번 학습함, mse = 0.19886076551796666
272 번 학습함, mse = 0.1987425100331807
273 번 학습함, mse = 0.19862452871132513
274 번 학습함, mse = 0.19850681561985603
275 번 학습함, mse = 0.19838936483566855
276 번 학습함, mse = 0.1982721704446117
277 번 학습함, mse = 0.19815522654100273
278 번 학습함, mse = 0.19803852722713713
279 번 학습함, mse = 0.19792206661279668
280 번 학습함, mse = 0.197805838814753
281 번 학습함, mse = 0.19768983795626655
282 번 학습함, mse = 0.19757405816657997
283 번 학습함, mse = 0.19745849358040607
284 번 학습함, mse = 0.1973431383374089
285 번 학습함, mse = 0.19722798658167712
286 번 학습함, mse = 0.19711303246119077
287 번 학습함, mse = 0.19699827012727855
288 번 학습함, mse = 0.19688369373406683
289 번 학습함, mse = 0.19676929743791863
290 번 학습함, mse = 0.19665507539686283
291 번 학습함, mse = 0.1965410217700128
292 번 학습

570 번 학습함, mse = 0.07804522458457656
571 번 학습함, mse = 0.0776560254611231
572 번 학습함, mse = 0.0772701342935534
573 번 학습함, mse = 0.07688751403688499
574 번 학습함, mse = 0.07650812818563339
575 번 학습함, mse = 0.07613194076099482
576 번 학습함, mse = 0.07575891629856771
577 번 학습함, mse = 0.0753890198365833
578 번 학습함, mse = 0.075022216904618
579 번 학습함, mse = 0.07465847351276061
580 번 학습함, mse = 0.07429775614121119
581 번 학습함, mse = 0.07394003173028593
582 번 학습함, mse = 0.07358526767080911
583 번 학습함, mse = 0.07323343179486905
584 번 학습함, mse = 0.07288449236691999
585 번 학습함, mse = 0.07253841807521097
586 번 학습함, mse = 0.07219517802352494
587 번 학습함, mse = 0.07185474172321124
588 번 학습함, mse = 0.07151707908549677
589 번 학습함, mse = 0.07118216041406078
590 번 학습함, mse = 0.0708499563978601
591 번 학습함, mse = 0.07052043810419201
592 번 학습함, mse = 0.07019357697198221
593 번 학습함, mse = 0.0698693448052871
594 번 학습함, mse = 0.0695477137669992
595 번 학습함, mse = 0.06922865637274553
596 번 학습함, mse = 0.06891214548497
597 번 학습함, m

879 번 학습함, mse = 0.02848801577056397
880 번 학습함, mse = 0.028425616153571764
881 번 학습함, mse = 0.028363470225587505
882 번 학습함, mse = 0.028301576479626923
883 번 학습함, mse = 0.028239933420467284
884 번 학습함, mse = 0.02817853956453465
885 번 학습함, mse = 0.02811739343979263
886 번 학습함, mse = 0.028056493585632097
887 번 학습함, mse = 0.027995838552762324
888 번 학습함, mse = 0.02793542690310325
889 번 학습함, mse = 0.02787525720967891
890 번 학습함, mse = 0.02781532805651225
891 번 학습함, mse = 0.027755638038520696
892 번 학습함, mse = 0.02769618576141337
893 번 학습함, mse = 0.027636969841589122
894 번 학습함, mse = 0.027577988906035716
895 번 학습함, mse = 0.027519241592230282
896 번 학습함, mse = 0.027460726548040708
897 번 학습함, mse = 0.02740244243162817
898 번 학습함, mse = 0.02734438791135075
899 번 학습함, mse = 0.02728656166566809
900 번 학습함, mse = 0.02722896238304709
901 번 학습함, mse = 0.02717158876186867
902 번 학습함, mse = 0.027114439510335465
903 번 학습함, mse = 0.027057513346380617
904 번 학습함, mse = 0.027000808997577525
905 번 학습함, mse = 0.02694

1187 번 학습함, mse = 0.016718514058348972
1188 번 학습함, mse = 0.016695342335191692
1189 번 학습함, mse = 0.016672230406456226
1190 번 학습함, mse = 0.016649178047879715
1191 번 학습함, mse = 0.016626185036304867
1192 번 학습함, mse = 0.016603251149673134
1193 번 학습함, mse = 0.016580376167017984
1194 번 학습함, mse = 0.01655755986845826
1195 번 학습함, mse = 0.01653480203519151
1196 번 학습함, mse = 0.01651210244948745
1197 번 학습함, mse = 0.016489460894681403
1198 번 학습함, mse = 0.016466877155167814
1199 번 학습함, mse = 0.01644435101639388
1200 번 학습함, mse = 0.016421882264853048
1201 번 학습함, mse = 0.016399470688078795
1202 번 학습함, mse = 0.016377116074638248
1203 번 학습함, mse = 0.016354818214125957
1204 번 학습함, mse = 0.016332576897157693
1205 번 학습함, mse = 0.016310391915364326
1206 번 학습함, mse = 0.01628826306138563
1207 번 학습함, mse = 0.016266190128864284
1208 번 학습함, mse = 0.016244172912439812
1209 번 학습함, mse = 0.016222211207742566
1210 번 학습함, mse = 0.016200304811387893
1211 번 학습함, mse = 0.01617845352097009
1212 번 학습함, mse = 0.01615665713

...

8572 번 학습함, mse = 0.0012775358514331297
8573 번 학습함, mse = 0.0012773683731730702
8574 번 학습함, mse = 0.0012772009379399512
8575 번 학습함, mse = 0.0012770335457173662
8576 번 학습함, mse = 0.0012768661964889208
8577 번 학습함, mse = 0.001276698890238221
8578 번 학습함, mse = 0.0012765316269488893
8579 번 학습함, mse = 0.0012763644066045485
8580 번 학습함, mse = 0.00127619722918884
8581 번 학습함, mse = 0.0012760300946853972
8582 번 학습함, mse = 0.0012758630030778781
8583 번 학습함, mse = 0.0012756959543499403
8584 번 학습함, mse = 0.001275528948485256
8585 번 학습함, mse = 0.0012753619854674939
8586 번 학습함, mse = 0.0012751950652803467
8587 번 학습함, mse = 0.0012750281879074976
8588 번 학습함, mse = 0.0012748613533326576
8589 번 학습함, mse = 0.0012746945615395274
8590 번 학습함, mse = 0.0012745278125118296
8591 번 학습함, mse = 0.0012743611062332868
8592 번 학습함, mse = 0.0012741944426876354
8593 번 학습함, mse = 0.0012740278218586123
8594 번 학습함, mse = 0.0012738612437299742
8595 번 학습함, mse = 0.001273694708285476
8596 번 학습함, mse = 0.0012735282155088847
8597 

8851 번 학습함, mse = 0.0012324212753799563
8852 번 학습함, mse = 0.0012322651938084126
8853 번 학습함, mse = 0.001232109150989973
8854 번 학습함, mse = 0.001231953146910359
8855 번 학습함, mse = 0.0012317971815552881
8856 번 학습함, mse = 0.0012316412549104937
8857 번 학습함, mse = 0.0012314853669617115
8858 번 학습함, mse = 0.0012313295176946866
8859 번 학습함, mse = 0.0012311737070951718
8860 번 학습함, mse = 0.0012310179351489208
8861 번 학습함, mse = 0.0012308622018417008
8862 번 학습함, mse = 0.001230706507159286
8863 번 학습함, mse = 0.001230550851087451
8864 번 학습함, mse = 0.0012303952336119857
8865 번 학습함, mse = 0.001230239654718681
8866 번 학습함, mse = 0.001230084114393335
8867 번 학습함, mse = 0.0012299286126217553
8868 번 학습함, mse = 0.0012297731493897553
8869 번 학습함, mse = 0.0012296177246831582
8870 번 학습함, mse = 0.0012294623384877866
8871 번 학습함, mse = 0.0012293069907894805
8872 번 학습함, mse = 0.0012291516815740745
8873 번 학습함, mse = 0.0012289964108274185
8874 번 학습함, mse = 0.001228841178535371
8875 번 학습함, mse = 0.0012286859846837906
8876 번 

9163 번 학습함, mse = 0.0011855349646870074
9164 번 학습함, mse = 0.0011853903144804384
9165 번 학습함, mse = 0.0011852456988886303
9166 번 학습함, mse = 0.0011851011178992859
9167 번 학습함, mse = 0.0011849565715001092
9168 번 학습함, mse = 0.0011848120596788167
9169 번 학습함, mse = 0.0011846675824231218
9170 번 학습함, mse = 0.00118452313972075
9171 번 학습함, mse = 0.001184378731559435
9172 번 학습함, mse = 0.0011842343579269053
9173 번 학습함, mse = 0.0011840900188109086
9174 번 학습함, mse = 0.001183945714199189
9175 번 학습함, mse = 0.001183801444079497
9176 번 학습함, mse = 0.0011836572084395942
9177 번 학습함, mse = 0.001183513007267246
9178 번 학습함, mse = 0.0011833688405502184
9179 번 학습함, mse = 0.0011832247082762873
9180 번 학습함, mse = 0.0011830806104332371
9181 번 학습함, mse = 0.0011829365470088552
9182 번 학습함, mse = 0.0011827925179909312
9183 번 학습함, mse = 0.0011826485233672663
9184 번 학습함, mse = 0.0011825045631256661
9185 번 학습함, mse = 0.0011823606372539377
9186 번 학습함, mse = 0.0011822167457398978
9187 번 학습함, mse = 0.0011820728885713675
9188 번

9435 번 학습함, mse = 0.0011474263250009318
9436 번 학습함, mse = 0.0011472906551804178
9437 번 학습함, mse = 0.0011471550168318761
9438 번 학습함, mse = 0.0011470194099444657
9439 번 학습함, mse = 0.0011468838345073467
9440 번 학습함, mse = 0.0011467482905096883
9441 번 학습함, mse = 0.0011466127779406626
9442 번 학습함, mse = 0.0011464772967894484
9443 번 학습함, mse = 0.001146341847045225
9444 번 학습함, mse = 0.0011462064286971837
9445 번 학습함, mse = 0.0011460710417345147
9446 번 학습함, mse = 0.001145935686146416
9447 번 학습함, mse = 0.0011458003619220889
9448 번 학습함, mse = 0.0011456650690507444
9449 번 학습함, mse = 0.0011455298075215886
9450 번 학습함, mse = 0.0011453945773238392
9451 번 학습함, mse = 0.0011452593784467256
9452 번 학습함, mse = 0.0011451242108794638
9453 번 학습함, mse = 0.0011449890746112943
9454 번 학습함, mse = 0.0011448539696314504
9455 번 학습함, mse = 0.0011447188959291712
9456 번 학습함, mse = 0.0011445838534937054
9457 번 학습함, mse = 0.0011444488423143066
9458 번 학습함, mse = 0.0011443138623802264
9459 번 학습함, mse = 0.0011441789136807296
94

9732 번 학습함, mse = 0.0011084704279094575
9733 번 학습함, mse = 0.0011083436492132172
9734 번 학습함, mse = 0.0011082168989747477
9735 번 학습함, mse = 0.001108090177184558
9736 번 학습함, mse = 0.0011079634838331643
9737 번 학습함, mse = 0.0011078368189110879
9738 번 학습함, mse = 0.0011077101824088487
9739 번 학습함, mse = 0.0011075835743169808
9740 번 학습함, mse = 0.0011074569946260123
9741 번 학습함, mse = 0.0011073304433264828
9742 번 학습함, mse = 0.0011072039204089322
9743 번 학습함, mse = 0.001107077425863906
9744 번 학습함, mse = 0.001106950959681953
9745 번 학습함, mse = 0.001106824521853625
9746 번 학습함, mse = 0.001106698112369483
9747 번 학습함, mse = 0.0011065717312200873
9748 번 학습함, mse = 0.0011064453783960044
9749 번 학습함, mse = 0.0011063190538878032
9750 번 학습함, mse = 0.0011061927576860601
9751 번 학습함, mse = 0.0011060664897813515
9752 번 학습함, mse = 0.0011059402501642599
9753 번 학습함, mse = 0.001105814038825375
9754 번 학습함, mse = 0.0011056878557552836
9755 번 학습함, mse = 0.0011055617009445838
9756 번 학습함, mse = 0.001105435574383873
9757 번 

In [12]:
#TEST
def Predict(data, net):
    numberOfdata = len(data)
    numberOfinp = len(data[0]) - 1
    for i in range(0, numberOfdata):
        Data = data[i]
        print(i + 1, '번째 input:', Forward_propagation(Data, net)[0][0:-1])
        predict_po = Forward_propagation(Data, net)[2]
        print('예측값에 대한 확률:', predict_po)
        predict_va = Predict_value(predict_po)
        print('예측값:', predict_va)
        print('정답:', Data[numberOfinp], '\n')


def Predict_value(pre_output):
    tmp = max(pre_output)
    index = pre_output.index(tmp)
    return index


# 데이타셋
dataset = [[3.5064385449265267, 2.34547092892632525, 0], [4.384621956392097, 3.4530853889904205, 0],
           [4.841442919897487, 4.02507852317520154, 0], [3.5985868973088437, 4.1621314217538705, 0],
           [2.887219775424049, 3.31523082529190005, 0], [9.79822645535526, 1.1052409596099566, 1],
           [7.8261241795117422, 0.6711054766067182, 1], [2.5026163932400305, 5.800780055043912, 1],
           [5.032436157202415, 8.650625621472184, 1], [4.095084253434162, 7.69104329159447, 1]]

# 트레이닝된 Weight값 입력
input
#Final_Weight = [[[4.1420949832869125, -3.1439422237080716, 2.053207587151659], [1.0227003808119506, -3.1849740173116396, -0.3659898981213347]], [[6.914969030301216, -6.892585888743422, -3.3776734299209576], [-6.9145405303614575, 6.892661536140577, 3.3773157954106936]]]
print("Input MODEL")
Final_Weight = eval(input())
print()
# 예측
Predict(dataset, Final_Weight)

Input MODEL
[[[0.13436424411240122, 0.8474337369372327, 0.763774618976614], [0.2550690257394217, 0.49543508709194095, 0.4494910647887381]], [[0.651592972722763, 0.7887233511355132, 0.0938595867742349], [0.02834747652200631, 0.8357651039198697, 0.43276706790505337]]] 

1 번째 input: [3.5064385449265267, 2.3454709289263254]
예측값에 대한 확률: [0.8099518614879568, 0.7742975348804565]
예측값: 0
정답: 0 

2 번째 input: [4.384621956392097, 3.4530853889904205]
예측값에 대한 확률: [0.8170717321097025, 0.7800821106594207]
예측값: 0
정답: 0 

3 번째 input: [4.841442919897487, 4.0250785231752015]
예측값에 대한 확률: [0.8189955641001906, 0.7817789016653995]
예측값: 0
정답: 0 

4 번째 input: [3.5985868973088437, 4.1621314217538705]
예측값에 대한 확률: [0.8181649975915356, 0.7808128376340171]
예측값: 0
정답: 0 

5 번째 input: [2.887219775424049, 3.3152308252919]
예측값에 대한 확률: [0.8142678241172914, 0.7772492446949237]
예측값: 0
정답: 0 

6 번째 input: [9.79822645535526, 1.1052409596099566]
예측값에 대한 확률: [0.8146579519417592, 0.7809087921984229]
예측값: 0
정답: 1 

7 번째 input: [

In [ ]:
#ReLu_Training
import math
import random

random.seed(1)


def MLP2(input, hidden, output):  # 네트워크 구성 및 weight 초기화
    net = list()
    h_layer = [[random.random() for i in range(input + 1)] for i in range(hidden)]
    net.append(h_layer)
    o_layer = [[random.random() for i in range(hidden + 1)] for i in range(output)]
    net.append(o_layer)
    return net


def Act_sigmoid(x):  # 활성화 함수 - 시그모이드
    y = 1 / (1 + math.exp(-x))
    return y


def Act_relu(x):
    if x > 0:
        y = x
    else:
        y = 0
    return y


def Weight_summation(data, net, layer):
    # 입력값에 layer를 사용한 이유는 히든층에서의 웨이트 계산할 때는 [0]네트워크(웨이트)
    #  아웃층에서 웨이트 계산할 때는 [1]네트워크(웨이트) 사용
    numberOfinp = len(net[0][0]) - 1  # 만약 인풋이 두개면 2
    numberOfhid = len(net[0])  # 히든노드 두개면 2
    numberOfout = len(net[1])  # 아웃노드 두개면 2
    W_s = []  # 웨이트 써메이션 초기화

    if layer == 'H':  # 인풋- > '히든'레이어 갈 때
        for i in range(0, numberOfhid):
            h_w = 0
            h_w = h_w + (1 * net[0][i][numberOfinp])  # 바이어스와 웨이트의 곱
            for j in range(0, numberOfinp):
                h_w = h_w + (data[j] * net[0][i][j])
            W_s.append(h_w)
        W_s = [W_s]

    if layer == 'O':
        for i in range(0, numberOfout):
            o_w = 0
            o_w = o_w + (1 * net[1][i][numberOfhid])
            for j in range(0, numberOfhid):
                o_w = o_w + data[j] * net[1][i][j]  # 히든노드
            W_s.append(o_w)
        W_s = [W_s]  # [ [ ~~ ]]
    return W_s


def Forward_propagation(data, net):  # 순전파 진행하면서 인풋데이타, 인풋-히든 웨이트
    node = list()
    node.append(data)
    Ws1 = Weight_summation(data, net, 'H')  # [[ h1_1_before,  h1_2_before ]]
    # print('[[h1_1_before,  h1_2_before]] : ',Ws1,'\n')
    numberOfhid = len(net[0])  # 히든노드 두개면 2
    numberOfout = len(net[1])

    H1 = list()
    for i in range(0, numberOfhid):
        H1.append(Act_relu(Ws1[0][i]))
    # H1 = [ H1_1 H1_2 ~~~~]
    node.append(H1)

    Ws2 = Weight_summation(H1, net, 'O')  # [[ o1_1_before, o_2_before ]]
    # print('[[ o1_1_before, o_2_before ]] : ',Ws2,'\n')

    O1 = list()
    for i in range(0, numberOfout):
        O1.append(Act_relu(Ws2[0][i]))

    node.append(O1)

    # print("[[IN1,IN2, T], [H1, H2], [OUT1, OUT2]]:",node,'\n')
    return node


def Backward_propagation_relu(weight, dataset):  # 역전파
    numberOfout = len(weight[1])  # 아웃노드 두개면 2
    numberOfdata = len(dataset)
    numberOfhid = len(weight[0])
    numberOfinp = len(weight[0][0]) - 1

    c2 = list()
    h_layer = [[0 for i in range(numberOfinp + 1)] for i in range(numberOfhid)]
    c2.append(h_layer)
    o_layer = [[0 for i in range(numberOfhid + 1)] for i in range(numberOfout)]
    c2.append(o_layer)
    c2

    init_back = c2
    for Da in range(0, numberOfdata):
        Data = dataset[Da]
        forward = Forward_propagation(Data, weight)
        Input = forward[0]
        Hid = forward[1]
        Output = forward[2]

        table = []
        for i in range(0, numberOfout):
            if i == Data[numberOfinp]:
                table.append(1)
            else:
                table.append(0)

        # 아웃에서 히든으로 역전파 과정
        for i in range(0, numberOfout):
            out_before = weight[1][i][numberOfhid] * 1  # 바이어스에 대한 역전파 시작
            for j in range(0, numberOfhid):
                out_before = out_before + weight[1][i][j] * Hid[j]
            if out_before > 0:
                g_prime_out_before = 1
            else:
                g_prime_out_before = 0

            init_back[1][i][numberOfhid] = init_back[1][i][numberOfhid] + \
                                           (2 * (Output[i] - table[i]) * g_prime_out_before * 1) * (1 / numberOfout)

            for j in range(0, numberOfhid):
                init_back[1][i][j] = init_back[1][i][j] * 1 + \
                                     (2 * (Output[i] - table[i]) * g_prime_out_before * Hid[j]) * (1 / numberOfout)

        # 히든에서 인풋으로 역전파 과정
        for i in range(0, numberOfhid):
            h_before = weight[0][i][numberOfinp] * 1
            for l in range(0, numberOfinp):
                h_before = h_before + weight[0][i][l] * Input[l]

            if h_before > 0:
                g_prime_h_before = 1
            else:
                g_prime_h_before = 0

            for m in range(0, numberOfout):
                out_before = weight[1][m][numberOfhid] * 1  # out_before의 초기값은 바이어스에 대한 웨이트
                for k in range(0, numberOfhid):
                    out_before = out_before + weight[1][m][k] * Hid[k]  # out1_before -> out2_before

                # 여기까지 이상없음
                if out_before > 0:
                    g_prime_out_before = 1
                else:
                    g_prime_out_before = 0

                init_back[0][i][numberOfinp] = init_back[0][i][numberOfinp] + \
                                               2 * ((Output[m] - table[m]) * g_prime_out_before * weight[1][m][i] \
                                                    * g_prime_h_before * 1) * (1 / numberOfout)  # 바이어스

                for p in range(0, numberOfinp):
                    init_back[0][i][p] = init_back[0][i][p] + \
                                         2 * ((Output[m] - table[m]) * g_prime_out_before * weight[1][m][i] \
                                              * g_prime_h_before * Input[p]) * (1 / numberOfout)

    for i in range(0, numberOfhid):
        for j in range(0, numberOfinp + 1):
            init_back[0][i][j] = init_back[0][i][j] / numberOfdata

    for i in range(0, numberOfout):
        for j in range(0, numberOfhid + 1):
            init_back[1][i][j] = init_back[1][i][j] / numberOfdata

    return init_back


def Weight_update(backpropa, Now_weight):
    numberOfout = len(Now_weight[1])  # 아웃노드 두개면 2
    numberOfhid = len(Now_weight[0])
    numberOfinp = len(Now_weight[0][0]) - 1

    New_weight = list()
    learning_rate = 0.01

    in_hid_w = Now_weight[0]
    hid_out_w = Now_weight[1]

    out_hid_back = backpropa[1]
    hid_in_back = backpropa[0]

    c2 = list()
    h_layer = [[0 for i in range(numberOfinp + 1)] for i in range(numberOfhid)]
    c2.append(h_layer)
    o_layer = [[0 for i in range(numberOfhid + 1)] for i in range(numberOfout)]
    c2.append(o_layer)

    # inp -> hid 업데이트
    for i in range(0, numberOfhid):
        c2[0][i][numberOfinp] = in_hid_w[i][numberOfinp] \
                                - (hid_in_back[i][numberOfinp] * learning_rate)
        for j in range(0, numberOfinp):
            c2[0][i][j] = in_hid_w[i][j] - (hid_in_back[i][j] * learning_rate)

    # hid -> out 업데이트
    for i in range(0, numberOfout):
        c2[1][i][numberOfhid] = hid_out_w[i][numberOfhid] \
                                - (out_hid_back[i][numberOfhid] * learning_rate)
        for j in range(0, numberOfhid):
            c2[1][i][j] = hid_out_w[i][j] - (out_hid_back[i][j] * learning_rate)

    return c2


def Loss_function(dataset, net):  # 손실함수 - MSE(평균 제곱 오차)
    numberOfdata = len(dataset)
    numberOfinp = len(net[0][0]) - 1
    numberOfout = len(net[1])
    mse = 0

    for j in range(0, numberOfdata):
        Data = dataset[j]
        table = []
        for i in range(0, numberOfout):
            if i == Data[numberOfinp]:
                table.append(1)
            else:
                table.append(0)

        Fpropa = Forward_propagation(Data, net)  # 아웃풋 구하기

        for j in range(0, numberOfout):
            mse = mse + (1 / numberOfout) * (Fpropa[2][j] - table[j]) ** 2

    mse = mse / numberOfdata
    return mse


def Epoch(dataset, net, N):  # N = 학습횟수

    updatedW = net

    for i in range(0, N):
        bp = Backward_propagation_relu(updatedW, dataset)
        updatedW = Weight_update(bp, updatedW)
        mse = Loss_function(dataset, updatedW)
        print(i + 1, '번 학습함, mse =', mse)
    print('최종 학습된 가중치:',updatedW)
    return updatedW


data = [[3.5064385449265267, 2.34547092892632525, 0], [4.384621956392097, 3.4530853889904205, 0],
        [4.841442919897487, 4.02507852317520154, 0], [3.5985868973088437, 4.1621314217538705, 0],
        [2.887219775424049, 3.31523082529190005, 0], [9.79822645535526, 1.1052409596099566, 1],
        [7.8261241795117422, 0.6711054766067182, 1], [2.5026163932400305, 5.800780055043912, 1],
        [5.032436157202415, 8.650625621472184, 1], [4.095084253434162, 7.69104329159447, 1]]
net = MLP2(2, 2, 2)

Epoch(data, net, 30000)